In [326]:
import pandas as pd
import numpy as np
import math
import talib
from datetime import date
import datetime
import time
import pandas_datareader.data as web
import statsmodels.tsa.stattools as ts
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
import dask.dataframe as dd

In [344]:
df = pd.read_csv('spxl_1min_30th.csv')
df = df[1162:1551]
df = df[['Date','Time', 'Close']]

In [345]:
df
# df=df.loc[(pd.to_datetime(df['Date']) > datetime.date(2017, 1, 1)) & (pd.to_datetime(df['Date']) < datetime.date(2020, 1, 1))]


,Date,Time,Close
1162,04/30/2019,9:31 AM,51.8491
1163,04/30/2019,9:32 AM,51.9000
1164,04/30/2019,9:33 AM,51.9200
1165,04/30/2019,9:34 AM,51.9300
1166,04/30/2019,9:35 AM,51.9400
1167,04/30/2019,9:36 AM,51.9300
1168,04/30/2019,9:37 AM,51.8900
1169,04/30/2019,9:38 AM,51.8900
1170,04/30/2019,9:39 AM,51.8800
1171,04/30/2019,9:40 AM,51.8000


In [333]:
shifted = df.shift()['Close']
df.insert(loc=1, column='Close Previous', value=shifted)
arr_df = np.array(df['Close'])
arr_df_prev = np.array(df['Close Previous'])

rsi = talib.RSI(arr_df, timeperiod=12)
rsi_prev = talib.RSI(arr_df_prev, timeperiod=12)

df.insert(loc=2, column='RSI', value=rsi)
df.insert(loc=3, column='RSI Previous', value=rsi_prev)

In [334]:
df.tail()
df = df.reset_index()
df.tail()

,index,Date,Close Previous,RSI,RSI Previous,Time,Close
384,1546,04/30/2019,52.2700,71.021492,64.007808,3:56 PM,52.3400
385,1547,04/30/2019,52.3400,68.907911,71.021492,3:57 PM,52.3299
386,1548,04/30/2019,52.3299,68.907911,68.907911,3:58 PM,52.3299
387,1549,04/30/2019,52.3299,68.907911,68.907911,3:59 PM,52.3299
388,1550,04/30/2019,52.3299,68.907911,68.907911,4:00 PM,52.3299


In [335]:
isLong = False
isShort = False
purchase_price = 0.0
portfolio = 10000

trades = {'time':[], 'close':[], 'trade signal':[]}
trades = pd.DataFrame(data=trades)

def rsi(row):
    global isLong, purchase_price, portfolio, trades
    
    if ((row['RSI'] <= 40.00) and (row['RSI Previous'] > 40.00) and (isLong == False)):
        purchase_price = row['Close']
        isLong = True
        trade = {'time':[row['Date']], 'close':[row['Close']], 'trade signal':["Buy"]}
        trade = pd.DataFrame(trade)
        trades = trades.append(trade)
            
    elif ((row['RSI'] >= 80.00) and (row['RSI Previous'] < 80.00) and (isLong == True)):
        portfolio = portfolio * (row['Close']/purchase_price)
        isLong = False
        trade = {'time':[row['Date']], 'close':[row['Close']], 'trade signal':["Sell"]}
        trade = pd.DataFrame(trade)
        trades = trades.append(trade)

backtest = df.apply(lambda row: rsi(row),axis=1)

In [336]:
trades_buy = trades.shift()['close']
trades['purchase price'] = trades_buy

def profit_loss(row):
    if (row['trade signal'] == "Sell"):
        p_l = (row['close'] / row['purchase price'])
        return p_l
p_l = trades.apply(lambda row: profit_loss(row),axis=1)
trades['profit/loss'] = p_l

In [337]:
trades

,close,time,trade signal,purchase price,profit/loss
0,51.7320,04/30/2019,Buy,NaN,NaN
0,52.2000,04/30/2019,Sell,51.732,1.009047
0,52.1401,04/30/2019,Buy,52.200,NaN


In [338]:
np.sum(np.array(trades['profit/loss']) >= 1, axis=0)

C:\Users\Public\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater_equal
  """Entry point for launching an IPython kernel.


1

In [339]:
np.sum(np.array(trades['profit/loss']) < 1, axis=0)

C:\Users\Public\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


0

In [340]:
len(trades)/2

1

In [341]:
trades['profit/loss'].mean()

1.0090466249130132

In [342]:
trades.sort_values('profit/loss', ascending=False).dropna()

,close,time,trade signal,purchase price,profit/loss
0,52.2,04/30/2019,Sell,51.732,1.009047


In [343]:
portfolio

10090.466249130131